## Python for data analysis (R users pay attention)

### How does climate affect prosperity?

We will look at the effect of climate through a proxy vatriable, the latitude. Prosperity will be measured by GDP per capita.

 * Look at GDP data for a number of cities around the world
 * Get the population and the latitude data for these cities
 * Compute the GDP per capita for each city
 * plot GDP per capita data against the latitude of each city

#### Reading data and down selecting

In [1]:
import pandas 
gdp = pandas.read_csv('https://raw.githubusercontent.com/pletzer/jupyterMoons/master/citiesGDP.csv')

In [19]:
# how many rows and columns in the table?

In [20]:
# show the title of each column

In [21]:
# show the first 10 rows

In [22]:
# select the cities for which the GDP > $1000bn

In [3]:
# read geographic data 
geo = pandas.read_csv('https://raw.githubusercontent.com/pletzer/swc-python-data/master/world_cities.csv')

In [24]:
# remove some columns

In [25]:
# get the new number of columns

### Table indexing

In [26]:
# show the first row

In [27]:
# display the city of the first row

### Masking

In [28]:
# list the cities in New Zealand with more than 100000 inhabitants

In [29]:
# sort the NZ cities by population

### clean up the data

In [30]:
# get all the cities named "London"

In [31]:
# get all the cities named "Portland" 

In [32]:
# remove some rows in the datasets

In [33]:
# check that London and Portland appear only once

In [34]:
# replace 'United States of America' -> 'USA'
# replace 'United Kingdom' -> 'UK'

### Merging tables

In [35]:
# combine the two tables
# merge the data frames on city and country

In [36]:
# calculate GDP per capita

In [37]:
# show cities with GDP per capita > $50k

### Visualize

In [38]:
# create scatter plot with size of population
% matplotlib inline
from matplotlib import pylab, pyplot
# nicer plot size
pylab.rcParams['figure.figsize'] = (12, 6)

pyplot.show()